In [1]:
import geopandas as gpd
import pandas as pd
import boto3

In [2]:
bucket_name = 'opera-validation-datasets'
root_folder_name = 'calval-database-dev/'

In [3]:
image_name = '20210912_034049_22_2421'

In [4]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')

In [5]:
collocation = gpd.read_file('Collocation_HLS_Planet_2021.geojson')

In [6]:
search = collocation[(collocation.pl_ID == image_name)].sort_values('hls_pl_AOI_intersect',ascending=False)
search = search.iloc[[0]]


In [7]:
new_image = search.drop(columns=['hls_Date','hls_Cloud_Cover','hls_Footprint','hls_AOI_intersect','hls_pl_AOI_intersect','chip_rID'])
new_image = new_image.rename(columns = {'hls_ID':'collocated_dswx','pl_Date':'datetime','pl_ID':'image_name','pl_Cloud_Cover':'cloud_cover','pl_AOI_intersect':'site_coverage','chip_ID':'site_name'})
new_image.head()

,collocated_dswx,datetime,image_name,cloud_cover,site_coverage,instrument,provider,resolution,timeDelta_days,site_name,geometry
20467,HLS.S30.T47ULQ.2021254T043701.v2.0,2021-09-12T03:40:49,20210912_034049_22_2421,0.0,97.608045,PSB.SD,planetscope,3.0,0.955574,4_8,"POLYGON ((97.44674 49.52974, 97.37427 49.34386..."


In [8]:
obj = s3.Object(bucket_name,root_folder_name+'image.geojson')
obj.get()

{'ResponseMetadata': {'RequestId': '5C0RR00YWQ9D1CAM',
  'HostId': 'ZcHv/W5PeXb11Lr6xwolpMJqFQLujnCPhClSuXxNQE+zl+2ZFWv5/i9AtYTwcXWYsS6MZDfo3/I=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZcHv/W5PeXb11Lr6xwolpMJqFQLujnCPhClSuXxNQE+zl+2ZFWv5/i9AtYTwcXWYsS6MZDfo3/I=',
   'x-amz-request-id': '5C0RR00YWQ9D1CAM',
   'date': 'Tue, 21 Jun 2022 17:15:07 GMT',
   'last-modified': 'Tue, 21 Jun 2022 16:53:21 GMT',
   'etag': '"2ea1b5b577635adba680e769e5b4597f"',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'server': 'AmazonS3',
   'content-length': '661'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2022, 6, 21, 16, 53, 21, tzinfo=tzutc()),
 'ContentLength': 661,
 'ETag': '"2ea1b5b577635adba680e769e5b4597f"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f8a30eae310>}

In [9]:
image_data = obj.get()['Body']
image_gdf = gpd.read_file(image_data)
image_gdf.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,site_coverage,site_name,timeDelta_days,geometry
0,0.0,HLS.S30.T15QYU.2021284T162221.v2.0,2021-10-11T15:54:55,20211011_155455_52_2262,PSB.SD,planetscope,3.0,70.640659,3_10,0.030674,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."


In [10]:
image_gdf_upd = image_gdf.append(new_image,ignore_index=True)
image_gdf_upd.head()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,cloud_cover,collocated_dswx,datetime,geometry,image_name,instrument,provider,resolution,site_coverage,site_name,timeDelta_days
0,0.0,HLS.S30.T15QYU.2021284T162221.v2.0,2021-10-11T15:54:55,"POLYGON ((-90.50851 16.35841, -90.54697 16.176...",20211011_155455_52_2262,PSB.SD,planetscope,3.0,70.640659,3_10,0.030674
1,0.0,HLS.S30.T47ULQ.2021254T043701.v2.0,2021-09-12T03:40:49,"POLYGON ((97.44674 49.52974, 97.37427 49.34386...",20210912_034049_22_2421,PSB.SD,planetscope,3.0,97.608045,4_8,0.955574


In [11]:
image_upd_bytes = bytes(image_gdf_upd.to_json(drop_id=True).encode('UTF-8'))
s3object = s3.Object(bucket_name,root_folder_name+'image.geojson')
s3object.put(Body=image_upd_bytes)

{'ResponseMetadata': {'RequestId': '82HJPQBCNKAG67RX',
  'HostId': 'NrvJNiZ6BeXO+k9p7eabRtjC4tdA7WaHDhs2eK0pXn787b7vVAywDhsGbrDaZi6E+NBLZ9VpZjs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NrvJNiZ6BeXO+k9p7eabRtjC4tdA7WaHDhs2eK0pXn787b7vVAywDhsGbrDaZi6E+NBLZ9VpZjs=',
   'x-amz-request-id': '82HJPQBCNKAG67RX',
   'date': 'Tue, 21 Jun 2022 17:15:42 GMT',
   'etag': '"4dde04f02a08554d35a07200496162db"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"4dde04f02a08554d35a07200496162db"'}

In [12]:
image_data = obj.get()['Body']
image_gdf = gpd.read_file(image_data)
image_gdf.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,site_coverage,site_name,timeDelta_days,geometry
0,0.0,HLS.S30.T15QYU.2021284T162221.v2.0,2021-10-11T15:54:55,20211011_155455_52_2262,PSB.SD,planetscope,3.0,70.640659,3_10,0.030674,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."
1,0.0,HLS.S30.T47ULQ.2021254T043701.v2.0,2021-09-12T03:40:49,20210912_034049_22_2421,PSB.SD,planetscope,3.0,97.608045,4_8,0.955574,"POLYGON ((97.44674 49.52974, 97.37427 49.34386..."
